# Enter Trip Information

In [2]:
# Enter a trip name (a unique identifier for the specified points of interest)
TRIP_NAME = "east"

# Enter your points of interest in the list below (include your starting location)
MY_POINTS_OF_INTEREST = [
    "San Francisco, California", # Starting Location
    "Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA", # Yosemite NP
    "D L Bliss State Park, California 89, South Lake Tahoe, CA", # Lake Tahoe
    "Zion National Park Visitor Center, Zion – Mount Carmel Highway, Hurricane, UT", # Zion NP
    "Arches National Park Visitor Center, Moab, UT", # Arches NP
    "Monument Valley Navajo Tribal Park, Main Monument Valley Road, Oljato-Monument Valley, AZ", # Monument Valley
    "Island in the Sky Visitor Center, Grand View Point Road, Moab, UT", # Canyonlands NP
    "Bryce Canyon National Park Visitor Center, Utah 63, Bryce, UT", # Bryce Canyon NP
    "Moro Rock Trail, California", # Sequoia NP
    "Horseshoe Bend Parking Lot, Page, AZ", # Horseshoe Bend Trail
    "Grand Canyon Visitor Center, South Entrance Road, Grand Canyon Village, AZ", # Grand Canyon NP
    "Calf Creek Campground, Boulder, UT", # Grand Staircase-Escalante NM
    "Red Cliffs Recreation Area, Unnamed Road, Washington, UT", # Red Cliffs Recreation Nature Trail
    "Natural Bridges Visitor Center, Natural Bridge, Lake Powell, UT", # Natural Bridges NM
    "Kanarra Creek Trailhead, Kanarraville, UT", # Kanarra Creek Canyon Trail
    "San Simeon, CA", # South Big Sur Drive
    "Big Sur, CA", # Middle Big Sur Drive
    "Carmel-by-the-Sea, CA", # North Big Sur Drive
    "Castle Rock Entrance Station Parking Lot, Unnamed Road, Saratoga, CA", # Saratoga Gap Trail
]

# Collect Driving Distance/Duration Information

In [3]:
from config import GOOGLE_MAPS_API_KEY

In [6]:
import googlemaps
from itertools import combinations

METERS_IN_MILE = 1609.34


def query_gmaps_api_for_one_way_driving_distance_and_duration(venue_names, google_maps_api_key):
    """
    Query Google Maps API for the one-way driving distance and duration
    between all combination pairs of venues from a list of venues.

    Parameters
    ----------
    venue_names : list
        A list of venue names.

    Returns
    -------
    distance_duration_data : dict
        A data dictionary with a list of venue 1 names, a list of venue
        2 names, a list of driving distances in miles, and a list of
        driving durations in seconds.

    Raises
    ------
    Exception : If the distance or duration between venue 1 and venue 2
    not found.

    """

    # Instantiate Google Maps API session
    gmaps = googlemaps.Client(google_maps_api_key)

    # Initialize data dictionary to hold values
    distance_duration_data = {
        'Venue 1': [],
        'Venue 2': [],
        'Distance (mi)': [],
        'Duration (s)': []
    }

    # Collect driving distance and duration data for each venue to venue combination
    for (venue_1, venue_2) in combinations(venue_names, 2):

        try:

            # Query Google Maps API for driving distance and duration
            trip = gmaps.distance_matrix(
                origins=[venue_1],
                destinations=[venue_2],
                mode="driving",
                units="metric")

            # Extract driving distance in meters and convert to miles
            distance_m = trip['rows'][0]['elements'][0]['distance']['value']
            distance_mi = round(distance_m / METERS_IN_MILE)

            # Extract driving duration in seconds
            duration_s = trip['rows'][0]['elements'][0]['duration']['value']

            # Add values to data dictionary
            distance_duration_data['Venue 1'].append(venue_1)
            distance_duration_data['Venue 2'].append(venue_2)
            distance_duration_data['Distance (mi)'].append(distance_mi)
            distance_duration_data['Duration (s)'].append(duration_s)

        except Exception:

            raise Exception("Error finding the distance between {} and {}.".format(venue_1, venue_2))

    return(distance_duration_data)

In [7]:
# Query Google Maps API for one-way driving distances and durations
distance_duration_data = query_gmaps_api_for_one_way_driving_distance_and_duration(MY_POINTS_OF_INTEREST, GOOGLE_MAPS_API_KEY)

In [8]:
distance_duration_data

{'Venue 1': ['San Francisco, California',
  'San Francisco, California',
  'San Francisco, California',
  'San Francisco, California',
  'San Francisco, California',
  'San Francisco, California',
  'San Francisco, California',
  'San Francisco, California',
  'San Francisco, California',
  'San Francisco, California',
  'San Francisco, California',
  'San Francisco, California',
  'San Francisco, California',
  'San Francisco, California',
  'San Francisco, California',
  'San Francisco, California',
  'San Francisco, California',
  'San Francisco, California',
  'Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA',
  'Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA',
  'Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA',
  'Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA',
  'Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA',
  'Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA',

In [9]:
import pandas as pd


def convert_seconds_to_hhmm(seconds):
    """
    Convert a value from seconds to hours and minutes in HH:MM format.

    Parameters
    ----------
    seconds : float or int
        The number of seconds.

    Returns
    -------
    hhmm : str
        The number of hours and minutes in HH:MM format.

    """

    # Determine number of full minutes and remainder seconds
    minutes, seconds = divmod(seconds, 60)

    # Determine number of full hours and remainder minutes
    hours, minutes = divmod(minutes, 60)

    # Convert hours and minutes to HH:MM format
    hhmm = "{:d}:{:02d}".format(hours, minutes)

    return(hhmm)


def create_distance_and_duration_df(distance_duration_data):
    """
    Create a distance and duration DataFrame from a distance and
    duration data dictionary.

    Parameters
    ----------
    distance_duration_data : dict
        A data dictionary with a list of venue 1 names, a list of venue
        2 names, a list of driving distances in miles, and a list of
        driving durations in seconds.

    Returns
    -------
    distance_duration_df : pandas.core.frame.DataFrame
        A distance and duration DataFrame including venue 1, venue 2,
        distance (mi), duration (s), and duration (hhmm).

    """

    distance_duration_df = pd.DataFrame(distance_duration_data)

    column_order = [
        'Venue 1',
        'Venue 2',
        'Distance (mi)',
        'Duration (s)']

    distance_duration_df = distance_duration_df[column_order]

    distance_duration_df['Duration (hhmm)'] = distance_duration_df['Duration (s)'].apply(convert_seconds_to_hhmm)

    return(distance_duration_df)

In [10]:
# Create DataFrame of one-way distances and durations
distance_duration_df = create_distance_and_duration_df(distance_duration_data)

In [11]:
distance_duration_df.head()

,Venue 1,Venue 2,Distance (mi),Duration (s),Duration (hhmm)
0,"San Francisco, California","Yosemite Valley Visitor Center, Village Drive,...",191,14181,3:56
1,"San Francisco, California","D L Bliss State Park, California 89, South Lak...",197,12542,3:29
2,"San Francisco, California","Zion National Park Visitor Center, Zion – Moun...",727,40019,11:06
3,"San Francisco, California","Arches National Park Visitor Center, Moab, UT",963,51219,14:13
4,"San Francisco, California","Monument Valley Navajo Tribal Park, Main Monum...",941,52195,14:29
